In [ ]:
import refinitiv.dataplatform as rdp
import ipywidgets as widgets
import ipysheet
import pandas as pd
import dateutil.parser
import calendar
import json
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import BMonthEnd
import configparser as cp

config = cp.ConfigParser()
config.read("config.cfg")
rdp.open_desktop_session(config['session']['app_key'])

#rdp.open_platform_session( config['session']['app_key'],
#    rdp.GrantPassword( username=config['session']['user'], password=config['session']['password'] ) 
#)

In [ ]:
rics = []
df = pd.DataFrame()

def create_tick_table(streaming_prices):
    global df,ticks_table
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    rics = snapshot['Instrument'].values
    df = pd.DataFrame(index=rics, columns=field_names)
    for ric in streaming_prices:
        for field_name, field_value in ric.get_fields().items():
            df.at[ric.name, field_name] = field_value
    ticks_table = ipysheet.from_dataframe(df)
    table_widget.children = [table_header, ticks_table]
    
def update_tick_table(streaming_prices, ric, fields):
    global df,ticks_table
    for field_name, field_value in fields.items():
        df.at[ric, field_name] = field_value
    ticks_table = ipysheet.from_dataframe(df)
    table_widget.children = [table_header, ticks_table]

def display_status(streaming_price, ric, status):
    status_str = "{}:{}".format( ric, status)
    status_output.value = status_str    


In [ ]:
fields = ['DSPLY_NAME','BID','ASK','BIDSIZE','ASKSIZE','TRDPRC_1','MID_PRICE']
drop_rics = ['VOD.L','BT.L','JPY=','EUR=','AUD=']

In [ ]:
def add_ric(value):
    global status_output,streaming_prices,rics
    rics.append(ric_drop_down.value)
    streaming_prices = rdp.StreamingPrices(
        universe = rics, 
        fields   = fields,
        on_complete = lambda streaming_prices : 
            create_tick_table(streaming_prices),
        on_update = lambda streaming_prices, instrument_name, fields : 
            update_tick_table(streaming_prices, instrument_name, fields),
        on_status = lambda streaming_price, instrument_name, status : 
            display_status(streaming_price, instrument_name, status)
    )
    streaming_prices.open()
    return

In [ ]:
ticks_table = ipysheet.sheet(rows=len(drop_rics),columns=len(fields),column_width=[10,10,10,10,10,10],column_headers=fields,row_headers=True)
ric_drop_down = widgets.Dropdown(description='RIC:', options=drop_rics, value='VOD.L', continuous_update=False, layout={'width':'200px'})
btn_add = widgets.Button(description='Add',layout={'width':'50px'})
btn_add.on_click(add_ric)

lbl_status = widgets.Label('  Status:', layout={'width':'50px'})
status_output = widgets.Label("N/A")


In [ ]:
table_header = widgets.HBox([ric_drop_down, btn_add,lbl_status,status_output])
table_widget = widgets.VBox([table_header, ticks_table],layout={'width':'900px'})
accordion_ticker = widgets.Accordion(children=[table_widget], layout={'width':'1400px'})
accordion_ticker.set_title(0, 'Realtime Tick updates')
ticker = widgets.VBox([accordion_ticker])
ticker